In [1]:
"""
CSTR (Continuously Stirred Tank Reactor) Environment Demo
========================================================

This demo showcases the CSTR environment from pc-gym, which simulates
a continuously stirred tank reactor for chemical process control.

The CSTR is a well-established model that's thoroughly tested and stable,
making it perfect for learning and experimentation.

State Variables (3 total):
- Ca: Concentration of reactant A (mol/L)
- T: Temperature (K)
- Cb: Concentration of reactant B (mol/L)

Action Variables (1 total):
- Tc: Coolant temperature (K)

Observations (3 total):
- Ca, T, Cb: Concentration A, Temperature, Concentration B
"""

import numpy as np
from pcgym import make_env
from rl.cstr.optimization.visualization import (
    plot_state_variables,
    plot_control_actions,
    plot_reward_evolution)
from rl.cstr.optimization.load_config_files import load_and_create_env_params
from rl.cstr.optimization.base_state_builder import denormalize_observations
from rl.cstr.optimization.base_action_adapter import denormalize_actions

In [2]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================

# Load configuration from YAML file
config_path = "/workspace/general_projects/rl_cstr_optimization/config/environments/cstr_environment.yaml"
env_params = load_and_create_env_params(config_path)

# Extract action space bounds from env_params for use in the notebook
a_space = env_params['a_space']
o_space = env_params['o_space']
nsteps = env_params['N']

print(f"Configuration loaded: {env_params}")
print(f"Action space: {a_space}")
print(f"Observation space: {o_space}")
print(f"Number of steps: {nsteps}")


Configuration loaded: {'N': 30, 'tsim': 26, 'SP': {'Ca': [0.85, 0.85, 0.85, 0.9, 0.9, 0.9, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87]}, 'o_space': {'low': array([  0.7, 300. ,   0.8]), 'high': array([  1. , 350. ,   0.9])}, 'a_space': {'low': array([295]), 'high': array([302])}, 'x0': array([  0.8, 330. ,   0.8]), 'r_scale': {'ca': 1000}, 'model': 'cstr', 'normalise_a': True, 'normalise_o': True, 'noise': True, 'integration_method': 'casadi', 'noise_percentage': 0.001}
Action space: {'low': array([295]), 'high': array([302])}
Observation space: {'low': array([  0.7, 300. ,   0.8]), 'high': array([  1. , 350. ,   0.9])}
Number of steps: 30


In [3]:
# Create the CSTR environment with proper parameters
# The environment simulates a continuously stirred tank reactor for chemical process control
env = make_env(env_params)

# Reset the environment to get initial state
# This returns the initial observation (concentrations at reactor outlet)
initial_observation, initial_info = env.reset()

print("=" * 60)
print(f"CSTR REACTOR DEMO - {nsteps} STEP SIMULATION")
print("=" * 60)
print(f"Initial observation (normalized): {initial_observation}")

# Denormalize the initial observation for better understanding
initial_real = denormalize_observations(initial_observation, o_space)

print(f"Initial observation (real values):")
print(f"  Ca: {initial_real[0]}")
print(f"  T:  {initial_real[1]}")
print(f"  Cb: {initial_real[2]}")

print(f"Observation shape: {initial_observation.shape}")
print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")
print("=" * 60)

CSTR REACTOR DEMO - 30 STEP SIMULATION
Initial observation (normalized): [-0.33333328  0.2        -1.00000024]
Initial observation (real values):
  Ca: 0.8000000079472857
  T:  330.0
  Cb: 0.7999999880790669
Observation shape: (3,)
Action space: Box(-1.0, 1.0, (1,), float32)
Observation space: Box(-1.0, 1.0, (3,), float32)


/workspace/general_projects/rl_cstr_optimization/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/workspace/general_projects/rl_cstr_optimization/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [7]:
# ============================================================================
# DATA STORAGE FOR ANALYSIS
# ============================================================================

# Lists to store data for visualization and analysis
observations = [initial_observation]  # Store all observations (concentrations at outlet)
actions = []       # Store all actions taken
rewards = []       # Store all rewards received
states = []        # Store full state information if available
denorm_actions = [] # Store denormalized actions
denorm_observations = [] # Store denormalized observations

In [8]:
# ============================================================================
# MAIN SIMULATION LOOP - nsteps STEPS
# ============================================================================

print(f"\nStarting {nsteps}-step simulation...")
print("-" * 40)

for step in range(nsteps):
    print(f"\nStep {step + 1}/{nsteps}:")

    # ========================================================================
    # ACTION SELECTION
    # ========================================================================
    
    # For this demo, we'll use a simple strategy:
    # - Moderate coolant temperature control
    # - Stay within the safe operating range
    
    # Since actions are normalized, we need to provide values between 0 and 1
    # These will be automatically scaled to the actual bounds defined in env_params
    action = np.array([
        0.5,    # Tc: Coolant temperature (normalized) - moderate value
    ])
    
    # Denormalize: actual_value = low + (normalized_value * (high - low))
    denorm_action = denormalize_actions(action, a_space)
    
    
    # ========================================================================
    # ENVIRONMENT STEP
    # ========================================================================
    
    # Execute the action in the environment
    # This advances the simulation by one time step
    # Returns: new_observation, reward, terminated, truncated, info
    observation, reward, terminated, truncated, info = env.step(action)
    
    # Denormalize observation for better understanding
    denorm_observation = denormalize_observations(observation, o_space)

    print(f" Normalized action: {action}")
    print(f" Denormalized action: {denorm_action}")
    print(f"  New observation (normalized): {observation}")
    print(f"  New observation (real values):")
    print(f"    Ca: {denorm_observation[0]}")
    print(f"    T:  {denorm_observation[1]}")
    print(f"    Cb: {denorm_observation[2]}")
    print(f"  Reward: {reward:.4f}")
    print(f"  Terminated: {terminated}")
    print(f"  Truncated: {truncated}")
    print(f"  observation type: {type(observation)}")
    print(f"  observation shape: {observation.shape}")
    print(f"  observation values: {observation}")
    print(f" reward type: {type(reward)}")
    print(f" reward shape: {reward.shape}")
    print(f" reward values: {reward}")
    print(f" terminated type: {type(terminated)}")
    print(f" terminated values: {terminated}")
    print(f" truncated type: {type(truncated)}")
    print(f" truncated values: {truncated}")
    print(f" info type: {type(info)}")
    print(f" info values: {info}")
    
    # ========================================================================
    # DATA STORAGE
    # ========================================================================
    
    # Store the data for later analysis
    observations.append(observation.copy())
    actions.append(action.copy())
    rewards.append(reward)
    denorm_actions.append(denorm_action.copy())
    denorm_observations.append(denorm_observation.copy())
    
    # ========================================================================
    # TERMINATION CHECK
    # ========================================================================
    
    # Check if the episode has ended
    if terminated or truncated:
        print(f"  Episode ended at step {step + 1}")
        break



Starting 30-step simulation...
----------------------------------------

Step 1/30:
 Normalized action: [0.5]
 Denormalized action: 298.5
  New observation (normalized): [ 0.14653181 -0.02924253  1.00000048]
  New observation (real values):
    Ca: 0.8719797722442464
    T:  324.2689367481782
    Cb: 0.9000000238418665
  Reward: -0.0008
  Terminated: False
  Truncated: False
  observation type: <class 'numpy.ndarray'>
  observation shape: (3,)
  observation values: [ 0.14653181 -0.02924253  1.00000048]
 reward type: <class 'numpy.float64'>
 reward shape: ()
 reward values: -0.000808509658379334
 terminated type: <class 'bool'>
 terminated values: False
 truncated type: <class 'bool'>
 truncated values: False
 info type: <class 'dict'>
 info values: {}

Step 2/30:
 Normalized action: [0.5]
 Denormalized action: 298.5
  New observation (normalized): [0.15940202 0.00413603 1.00000048]
  New observation (real values):
    Ca: 0.8739103034593626
    T:  325.10340065309236
    Cb: 0.9000000

In [21]:
# ============================================================================
# SIMULATION COMPLETE - ANALYSIS AND VISUALIZATION
# ============================================================================

print("\n" + "=" * 60)
print("SIMULATION COMPLETE - ANALYSIS")
print("=" * 60)

# Convert lists to numpy arrays for easier analysis
observations = np.array(observations)
actions = np.array(actions)
rewards = np.array(rewards)
denorm_actions = np.array(denorm_actions)
denorm_observations = np.array(denorm_observations)

print(f"Total steps completed: {len(observations)}")
print(f"Average reward: {np.mean(rewards):.4f}")
print(f"Total reward: {np.sum(rewards):.4f}")
print(f"Denormalized actions: {denorm_actions}")
print(f"Denormalized observations: {denorm_observations}")


SIMULATION COMPLETE - ANALYSIS
Total steps completed: 30
Average reward: -0.0001
Total reward: -0.0034
Denormalized actions: [298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5
 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5 298.5
 298.5 298.5 298.5 298.5 298.5]
Denormalized observations: [[  0.82933053 326.2351389    0.85000001]
 [  0.85320202 325.72286714   0.85000001]
 [  0.86592612 325.32572191   0.85000001]
 [  0.87185123 324.85448049   0.90000002]
 [  0.87309998 325.18705228   0.90000002]
 [  0.87386542 324.91336884   0.90000002]
 [  0.87358614 325.23810699   0.87000001]
 [  0.87316031 324.80039452   0.87000001]
 [  0.87297796 324.62494774   0.87000001]
 [  0.87523312 324.88222768   0.87000001]
 [  0.87235698 324.78689364   0.87000001]
 [  0.87455333 324.56197494   0.87000001]
 [  0.87373117 325.03661682   0.87000001]
 [  0.87296204 324.88307285   0.87000001]
 [  0.87292444 324.7265825    0.87000001]
 [  0.87290013 325.05845825   0.8700000

In [22]:
# ============================================================================
# VISUALIZATION
# ============================================================================

In [ ]:
# Create all four plots
variable_definitions = [
    {
        'name': 'Ca (Concentration A)',
        'index': 0,
        'color': 'blue',
        'symbol': 'circle',
        'yaxis_title': 'Concentration (mol/L)'
    },
    {
        'name': 'T (Temperature)',
        'index': 1,
        'color': 'red',
        'symbol': 'square',
        'yaxis_title': 'Temperature (K)'
    },
    {
        'name': 'Cb (Concentration B)',
        'index': 2,
        'color': 'green',
        'symbol': 'triangle-up',
        'yaxis_title': 'Concentration (mol/L)'
    }
]

plot_state_variables(denorm_observations, variable_definitions)

In [ ]:
# Create all four plots
plot_control_actions(denorm_actions)


In [ ]:
plot_reward_evolution(rewards)

In [23]:
state, info = env.reset()

/workspace/general_projects/rl_cstr_optimization/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:236: UserWarning:

WARN: Box low's precision lowered by casting to float32, current low.dtype=float64

/workspace/general_projects/rl_cstr_optimization/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:306: UserWarning:

WARN: Box high's precision lowered by casting to float32, current high.dtype=float64



In [24]:
state

array([-0.33333328,  0.2       , -1.00000024])

In [25]:
info

{'r_init': 0}

In [ ]:
# Close the environment to free resources
env.close()